In [1]:
import pickle

# Load the pickle file
with open('Vectordatabase/financial_chunks_metadata_sentences.pkl', 'rb') as file:
    text_chunks = pickle.load(file)

In [2]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# Initialize SentenceTransformer model
text_embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Load the existing FAISS index
index = faiss.read_index('Vectordatabase/financial_docs_text_index_sentences.faiss')

def retrieve_similar(query, k=1):
    # Get query embedding
    query_embedding = text_embedder.encode([query])
    # Search in FAISS index
    distances, indices = index.search(query_embedding.astype(np.float32), k)
    return indices, distances

c:\Users\David\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
import os

from groq import Groq
print(os.environ)
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

environ({'ALLUSERSPROFILE': 'C:\\ProgramData', 'APPDATA': 'C:\\Users\\David\\AppData\\Roaming', 'CHOCOLATEYINSTALL': 'C:\\ProgramData\\chocolatey', 'CHOCOLATEYLASTPATHUPDATE': '133699717484554630', 'CHROME_CRASHPAD_PIPE_NAME': '\\\\.\\pipe\\crashpad_9780_GCVZIMYTWWVVPJFL', 'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files', 'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files', 'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files', 'COMPUTERNAME': 'DESKTOP-O3EMPAD', 'COMSPEC': 'C:\\Windows\\system32\\cmd.exe', 'DRIVERDATA': 'C:\\Windows\\System32\\Drivers\\DriverData', 'ELECTRON_RUN_AS_NODE': '1', 'GOPATH': 'C:\\Users\\David\\go', 'GROQ_API_KEY': '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', 'HOMEDRIVE': 'C:', 'HOMEPATH': '\\Users\\David', 'JPY_INTERRUPT_EVENT': '1892', 'LOCALAPPDATA': 'C:\\Users\\David\\AppData\\Local', 'LOGONSERVER': '\\\\DESKTOP-O3EMPAD', 'NUMBER_OF_PROCESSORS': '8', 'ONEDRIVE': 'C:\\Users\\David\\OneDrive', 'ORIGINAL_XDG_CURRENT_DE

In [10]:
def generate_response(query, context):
    
    input_text = f"question: {query} context: {context}. Please limit the response to 5 sentences, and use visual language to explain the concept. No images, and no newlines just a small paragraph."


    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": input_text,
            }
        ],
        model="llama3-8b-8192",
    )

    response = chat_completion.choices[0].message.content

    return response

In [11]:
import pandas as pd
df = pd.read_csv('LLM_test2.csv')

In [12]:
import time
# Loop through the DataFrame, processing 30 rows every minute
for start_idx in range(0, len(df), 30):
    end_idx = min(start_idx + 30, len(df))  # Ensure we don't go out of bounds
    
    # Apply the function to the current chunk using both columns
    df.loc[start_idx:end_idx-1, 'RAG_5_sent_visual'] = df.loc[start_idx:end_idx-1].apply(
        lambda row: generate_response(row['Question'], row['RAG_context']), axis=1
    )
    
    # Print progress for tracking
    print(f"Processed rows {start_idx} to {end_idx - 1}")
    
    # Sleep for 60 seconds after processing each chunk
    if end_idx < len(df):  # No need to sleep after the last chunk
        print("Sleeping for 1 minute...")
        time.sleep(60)

# Display the result
print(df)

Processed rows 0 to 29
Sleeping for 1 minute...
Processed rows 30 to 59
Sleeping for 1 minute...
Processed rows 60 to 61
    Unnamed: 0.1  Unnamed: 0  \
0              0           0   
1              1           1   
2              2           2   
3              3           3   
4              4           4   
..           ...         ...   
57            57          57   
58            58          58   
59            59          59   
60            60          60   
61            61          61   

                                             Question  \
0   What is Portillo’s business model and what typ...   
1   How many restaurants did Portillo’s operate as...   
2   What are Portillo’s growth plans for restauran...   
3   What impact did inflation have on Portillo’s f...   
4   How does Portillo’s plan to maintain profitabi...   
..                                                ...   
57  What initiatives has Portillo’s implemented to...   
58  How does Portillo’s address potent

In [13]:
df.to_csv('LLM_test4.csv', index=False)